<div class=title>
<strong><center><font size="3">DAY 20 - IYKRA</font></center>
<hr>
<center><font size="5"><strong>Data Science Bootcamp</strong></font></center>
<center><font size="4"><strong>Applied Machine Learning: Marketing</strong></font></center>
<hr>
<p style="text-align:center">Author</p>
<center>Joshua Effendi</center>
<p style="text-align:center">Date:</p>
<center>15 November 2019</center></strong>
</div>

In [11]:
import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import HTML

def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

<div class=title1>
    <b><font size='4'>Apriori</font></b>
</div>

<div class=title2>
    <b><font size='3'>Import Libraries</font></b>
</div>

In [4]:
from collections import defaultdict
import csv

<div class=title2>
    <b><font size='3'>Define Apriori Class</font></b>
</div>

In [5]:
class Apriori(object):
    def __init__(self, minSupp, minConf):
        """ Parameters setting
        """
        self.minSupp = minSupp  # min support (used for mining frequent sets)
        self.minConf = minConf  # min confidence (used for mining association rules)

    def fit(self, filePath):
        """ Run the apriori algorithm, return the frequent *-term sets. 
        """
        # Initialize some variables to hold the tmp result
        transListSet  = self.getTransListSet(filePath)   # get transactions (list that contain sets)
        itemSet       = self.getOneItemSet(transListSet) # get 1-item set
        itemCountDict = defaultdict(int)         # key=candiate k-item(k=1/2/...), value=count
        freqSet       = dict()                   # a dict store all frequent *-items set
        
        self.transLength = len(transListSet)     # number of transactions
        self.itemSet     = itemSet
        
        # Get the frequent 1-term set
        freqOneTermSet = self.getItemsWithMinSupp(transListSet, itemSet, 
                                             itemCountDict, self.minSupp)

        # Main loop
        k = 1
        currFreqTermSet = freqOneTermSet
        while currFreqTermSet != set():
            freqSet[k] = currFreqTermSet  # save the result
            k += 1
            currCandiItemSet = self.getJoinedItemSet(currFreqTermSet, k) # get new candiate k-terms set
            currFreqTermSet  = self.getItemsWithMinSupp(transListSet, currCandiItemSet, 
                                                   itemCountDict, self.minSupp) # frequent k-terms set
            
            
        #
        self.itemCountDict = itemCountDict # 所有候选项以及出现的次数(不仅仅是频繁项),用来计算置信度啊
        self.freqSet       = freqSet       # Only frequent items(a dict: freqSet[1] indicate frequent 1-term set )
        return itemCountDict, freqSet
            
            
    def getSpecRules(self, rhs):
        """ Specify a right item, construct rules for it
        """
        if rhs not in self.itemSet:
            print('Please input a term contain in the term-set !')
            return None
        
        rules = dict()
        for key, value in self.freqSet.items():
            for item in value:
                if rhs.issubset(item) and len(item) > 1:
                    item_supp = self.getSupport(item)
                    item = item.difference(rhs)
                    conf = item_supp / self.getSupport(item)
                    if conf >= self.minConf:
                        rules[item] = conf
        return rules
        
    
    def getSupport(self, item):
        """ Get the support of item """
        return self.itemCountDict[item] / self.transLength
        
        
    def getJoinedItemSet(self, termSet, k):
        """ Generate new k-terms candiate itemset"""
        return set([term1.union(term2) for term1 in termSet for term2 in termSet 
                    if len(term1.union(term2))==k])
    
        
    def getOneItemSet(self, transListSet):
        """ Get unique 1-item set in `set` format 
        """
        itemSet = set()
        for line in transListSet:
            for item in line:
                itemSet.add(frozenset([item]))
        return itemSet
        
    
    def getTransListSet(self, filePath):
        """ Get transactions in list format 
        """
        transListSet = []
        with open(filePath, 'r') as file:
            reader = csv.reader(file, delimiter=',')
            for line in reader:
                transListSet.append(set(line))                
        return transListSet
                
    
    def getItemsWithMinSupp(self, transListSet, itemSet, freqSet, minSupp):
        """ Get frequent item set using min support
        """
        itemSet_  = set()
        localSet_ = defaultdict(int)
        for item in itemSet:
            freqSet[item]   += sum([1 for trans in transListSet if item.issubset(trans)])
            localSet_[item] += sum([1 for trans in transListSet if item.issubset(trans)])
        
        # Only conserve frequent item-set 
        n = len(transListSet)
        for item, cnt in localSet_.items():
            itemSet_.add(item) if float(cnt)/n >= minSupp else None
        
        return itemSet_

<div class=title2>
    <b><font size='3'>Calculate Apriori (Association)</font></b>
</div>

In [15]:
# Get two important parameters
filePath = './data/transaction_hci.csv'
minSupp  = 0.15
minConf  = 0.5
rhs      = frozenset(['Backpack'])
print("""Parameters: \n - filePath: {} \n - mininum support: {} \n - mininum confidence: {} \n - rhs: {}\n""".\
      format(filePath,minSupp,minConf, rhs))

# Run and print
objApriori = Apriori(minSupp, minConf)
itemCountDict, freqSet = objApriori.fit(filePath)
for key, value in freqSet.items():
    print('frequent {}-term set:'.format(key))
    print('-'*20)
    for itemset in value:
        print(list(itemset))
    print()

# Return rules with regard of `rhs`
rules = objApriori.getSpecRules(rhs)
print('-'*20)
print('rules refer to {}'.format(list(rhs)))
for key, value in rules.items():
    print('{} -> {}: {}'.format(list(key), list(rhs), value))

Parameters: 
 - filePath: ./data/transaction_hci.csv 
 - mininum support: 0.15 
 - mininum confidence: 0.5 
 - rhs: frozenset({'Backpack'})

frequent 1-term set:
--------------------
['FitBit']
['TP-Link']
['Nivea']
['Mouse']
['GoPro']
['Yonex']
['Music-Sampler']
['Guitar']
['Xiaomi']
['Backpack']

frequent 2-term set:
--------------------
['TP-Link', 'Mouse']
['Backpack', 'Guitar']
['Backpack', 'Xiaomi']
['GoPro', 'FitBit']
['Mouse', 'Backpack']

--------------------
rules refer to ['Backpack']
['Guitar'] -> ['Backpack']: 0.5384615384615384
['Xiaomi'] -> ['Backpack']: 0.5


<div class=title1>
    <b><font size='4'>Collaborative Filtering</font></b>
</div>

<div class=title2>
    <b><font size='3'>aryankashyap0</font></b>
</div>

<div class=title2>
    <b><font size='3'>Define Dataset</font></b>
</div>

In [16]:
dataset={'Alice': {'Love At Last': 5, 
                   'Romance Forever': 5,
                   'Cute puppies of love': 0, 
                   'Nonstop car chases': 0, 
                   'Swords vs karate': 0},
         
         'Bob': {'Love At Last': 5,
                 'Romance Forever': 0,
                 'Cute puppies of love': 4,
                 'Nonstop car chases': 0,
                 'Swords vs karate': 0},
         
         'Carol': {'Love At Last': 0, 
                   'Romance Forever': 0,
                   'Cute puppies of love': 0,
                   'Nonstop car chases': 5,
                   'Swords vs karate': 5},
         
         'Dave': {'Love At Last': 0, 
                  'Romance Forever': 0,
                  'Cute puppies of love': 0,
                  'Nonstop car chases': 4,
                  'Swords vs karate': 0}
        }

<div class=title2>
    <b><font size='3'>Define Similarity Score</font></b>
</div>

In [23]:
from math import sqrt

def similarity_score(person1,person2):

    # this Returns the ration euclidean distancen score of person 1 and 2

    # To get both rated items by person 1 and 2
    both_viewed = {}

    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1
        
        # The Conditions to check if they both have common rating items
        if len(both_viewed) == 0:
            return 0

        # Finding Euclidean distance
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item], 2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)
        
        return 1/(1+sqrt(sum_of_eclidean_distance))

In [24]:
print(similarity_score('Alice','Bob'))

0.13507810593582123


<div class=title2>
    <b><font size='3'>Define Person Correlation</font></b>
</div>

In [25]:
def person_correlation(person1, person2):

   # To get both rated items
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)

    # Checking for ratings in common
    if number_of_ratings == 0:
        return 0

    # Add up all the preferences of each user
    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    # Calculate the pearson score
    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))

    if denominator_value == 0:
        return 0
    else:
        r = numerator_value / denominator_value
        return r

In [26]:
print(person_correlation('Alice','Bob'))

0.2566324512873683


<div class=title2>
    <b><font size='3'>Define Most Similar Users</font></b>
</div>

In [27]:
def most_similar_users(person, number_of_users):

    # returns the number_of_users (similar persons) for a given specific person
    scores = [(person_correlation(person, other_person), other_person) for other_person in dataset if other_person != person]

    # Sort the similar persons so the highest scores person will appear at the first
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]

In [29]:
print(most_similar_users('Alice',3))

[(0.2566324512873683, 'Bob'), (-0.4082482904638631, 'Dave'), (-0.6666666666666666, 'Carol')]


<div class=title2>
    <b><font size='3'>Define User Recommendations</font></b>
</div>

In [30]:
def user_recommendations(person):

    # Gets recommendations for a person by using a weighted average of every other user's rankings
    totals = {}
    simSums = {}
    rankings_list =[]
    for other in dataset:
        # don't compare me to myself
        if other == person:
            continue
        sim = person_correlation(person,other)
        #print ">>>>>>>",sim

        # ignore scores of zero or lower
        if sim <=0: 
            continue
        for item in dataset[other]:

            # only score movies i haven't seen yet
            if item not in dataset[person] or dataset[person][item] == 0:

            # Similrity * score
                totals.setdefault(item,0)
                totals[item] += dataset[other][item]* sim
                # sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+= sim

        # Create the normalized list

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    rankings.reverse()
    # returns the recommended items
    recommendataions_list = [recommend_item for score, recommend_item in rankings]
    return recommendataions_list

In [31]:
print(user_recommendations('Alice'))

['Cute puppies of love', 'Swords vs karate', 'Nonstop car chases']


<div class=title2>
    <b><font size='3'>revantkumar</font></b>
</div>

<div class=title2>
    <b><font size='3'>User Based</font></b>
</div>

In [43]:
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
#from sklearn.utils.extmath import np.dot

users = 6040
items = 3952

def readingFile(filename):
	f = open(filename,"r")
	data = []
	for row in f:
		r = row.split(',')
		e = [int(r[0]), int(r[1]), int(r[2])]
		data.append(e)
	return data

def similarity_user(data):
	print ("Hello User")
	#f_i_d = open("sim_user_based.txt","w")
	user_similarity_cosine = np.zeros((users,users))
	user_similarity_jaccard = np.zeros((users,users))
	user_similarity_pearson = np.zeros((users,users))
	for user1 in range(users):
		print (user1)
		for user2 in range(users):
			if np.count_nonzero(data[user1]) and np.count_nonzero(data[user2]):
				user_similarity_cosine[user1][user2] = 1-scipy.spatial.distance.cosine(data[user1],data[user2])
				user_similarity_jaccard[user1][user2] = 1-scipy.spatial.distance.jaccard(data[user1],data[user2])
				try:
					if not math.isnan(scipy.stats.pearsonr(data[user1],data[user2])[0]):
						user_similarity_pearson[user1][user2] = scipy.stats.pearsonr(data[user1],data[user2])[0]
					else:
						user_similarity_pearson[user1][user2] = 0
				except:
					user_similarity_pearson[user1][user2] = 0

			#f_i_d.write(str(user1) + "," + str(user2) + "," + str(user_similarity_cosine[user1][user2]) + "," + str(user_similarity_jaccard[user1][user2]) + "," + str(user_similarity_pearson[user1][user2]) + "\n")
	#f_i_d.close()
	return user_similarity_cosine, user_similarity_jaccard, user_similarity_pearson

def crossValidation(data):
	k_fold = KFold(n_splits=10)

	Mat = np.zeros((users,items))
	for e in data:
		Mat[e[0]-1][e[1]-1] = e[2]

	sim_user_cosine, sim_user_jaccard, sim_user_pearson = similarity_user(Mat)
	#sim_user_cosine, sim_user_jaccard, sim_user_pearson = np.random.rand(users,users), np.random.rand(users,users), np.random.rand(users,users)

	'''sim_user_cosine = np.zeros((users,users))
	sim_user_jaccard = np.zeros((users,users))
	sim_user_pearson = np.zeros((users,users))

	f_sim = open("sim_user_based.txt", "r")
	for row in f_sim:
		r = row.strip().split(',')
		sim_user_cosine[int(r[0])][int(r[1])] = float(r[2])
		sim_user_jaccard[int(r[0])][int(r[1])] = float(r[3])
		sim_user_pearson[int(r[0])][int(r[1])] = float(r[4])
	f_sim.close()'''

	rmse_cosine = []
	rmse_jaccard = []
	rmse_pearson = []

	for train_indices, test_indices in k_fold:
		train = [data[i] for i in train_indices]
		test = [data[i] for i in test_indices]

		M = np.zeros((users,items))

		for e in train:
			M[e[0]-1][e[1]-1] = e[2]

		true_rate = []
		pred_rate_cosine = []
		pred_rate_jaccard = []
		pred_rate_pearson = []

		for e in test:
			user = e[0]
			item = e[1]
			true_rate.append(e[2])

			pred_cosine = 3.0
			pred_jaccard = 3.0
			pred_pearson = 3.0

			#user-based
			if np.count_nonzero(M[user-1]):
				sim_cosine = sim_user_cosine[user-1]
				sim_jaccard = sim_user_jaccard[user-1]
				sim_pearson = sim_user_pearson[user-1]
				ind = (M[:,item-1] > 0)
				#ind[user-1] = False
				normal_cosine = np.sum(np.absolute(sim_cosine[ind]))
				normal_jaccard = np.sum(np.absolute(sim_jaccard[ind]))
				normal_pearson = np.sum(np.absolute(sim_pearson[ind]))
				if normal_cosine > 0:
					pred_cosine = np.dot(sim_cosine,M[:,item-1])/normal_cosine

				if normal_jaccard > 0:
					pred_jaccard = np.dot(sim_jaccard,M[:,item-1])/normal_jaccard

				if normal_pearson > 0:
					pred_pearson = np.dot(sim_pearson,M[:,item-1])/normal_pearson

			if pred_cosine < 0:
				pred_cosine = 0

			if pred_cosine > 5:
				pred_cosine = 5

			if pred_jaccard < 0:
				pred_jaccard = 0

			if pred_jaccard > 5:
				pred_jaccard = 5

			if pred_pearson < 0:
				pred_pearson = 0

			if pred_pearson > 5:
				pred_pearson = 5

			print (str(user) + "\t" + str(item) + "\t" + str(e[2]) + "\t" + str(pred_cosine) + "\t" + str(pred_jaccard) + "\t" + str(pred_pearson))
			pred_rate_cosine.append(pred_cosine)
			pred_rate_jaccard.append(pred_jaccard)
			pred_rate_pearson.append(pred_pearson)

		rmse_cosine.append(sqrt(mean_squared_error(true_rate, pred_rate_cosine)))
		rmse_jaccard.append(sqrt(mean_squared_error(true_rate, pred_rate_jaccard)))
		rmse_pearson.append(sqrt(mean_squared_error(true_rate, pred_rate_pearson)))

		print (str(sqrt(mean_squared_error(true_rate, pred_rate_cosine))) + "\t" + str(sqrt(mean_squared_error(true_rate, pred_rate_jaccard))) + "\t" + str(sqrt(mean_squared_error(true_rate, pred_rate_pearson))))
		#raw_input()

	#print sum(rms) / float(len(rms))
	rmse_cosine = sum(rmse_cosine) / float(len(rmse_cosine))
	rmse_pearson = sum(rmse_pearson) / float(len(rmse_pearson))
	rmse_jaccard = sum(rmse_jaccard) / float(len(rmse_jaccard))

	print (str(rmse_cosine) + "\t" + str(rmse_jaccard) + "\t" + str(rmse_pearson))

	f_rmse = open("rmse_user.txt","w")
	f_rmse.write(str(rmse_cosine) + "\t" + str(rmse_jaccard) + "\t" + str(rmse_pearson) + "\n")

	rmse = [rmse_cosine, rmse_jaccard, rmse_pearson]
	req_sim = rmse.index(min(rmse))

	print (req_sim)
	f_rmse.write(str(req_sim))
	f_rmse.close()

	if req_sim == 0:
		sim_mat_user = sim_user_cosine

	if req_sim == 1:
		sim_mat_user = sim_user_jaccard

	if req_sim == 2:
		sim_mat_user = sim_user_pearson

	#predictRating(Mat, sim_mat_user)
	return Mat, sim_mat_user


def predictRating(recommend_data):

	M, sim_user = crossValidation(recommend_data)

	#f = open("toBeRated.csv","r")
	f = open(sys.argv[2],"r")
	toBeRated = {"user":[], "item":[]}
	for row in f:
		r = row.split(',')	
		toBeRated["item"].append(int(r[1]))
		toBeRated["user"].append(int(r[0]))

	f.close()

	pred_rate = []

	#fw = open('result1.csv','w')
	fw_w = open('result1.csv','w')

	l = len(toBeRated["user"])
	for e in range(l):
		user = toBeRated["user"][e]
		item = toBeRated["item"][e]

		pred = 3.0

		#user-based
		if np.count_nonzero(M[user-1]):
			sim = sim_user[user-1]
			ind = (M[:,item-1] > 0)
			#ind[user-1] = False
			normal = np.sum(np.absolute(sim[ind]))
			if normal > 0:
				pred = np.dot(sim,M[:,item-1])/normal

		if pred < 0:
			pred = 0

		if pred > 5:
			pred = 5

		pred_rate.append(pred)
		print (str(user) + "," + str(item) + "," + str(pred))
		#fw.write(str(user) + "," + str(item) + "," + str(pred) + "\n")
		fw_w.write(str(pred) + "\n")

	#fw.close()
	fw_w.close()

recommend_data = readingFile("ratings.csv")
# recommend_data = readingFile(sys.argv[1])
# recommend_data = readingFile(['ratings.csv', 'toBeRated.csv'])
#crossValidation(recommend_data)
predictRating(recommend_data)

FileNotFoundError: [Errno 2] No such file or directory: 'ratings.csv,toBeRated.csv'

<div class=title2>
    <b><font size='3'>Item Based</font></b>
</div>

In [41]:
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
#from sklearn.utils.extmath import np.dot

users = 6040
items = 3952

def readingFile(filename):
	f = open(filename,"r")
	data = []
	for row in f:
		r = row.split(',')
		e = [int(r[0]), int(r[1]), int(r[2])]
		data.append(e)
	return data

def similarity_item(data):
	print("Hello")
	#f_i_d = open("sim_item_based.txt","w")
	item_similarity_cosine = np.zeros((items,items))
	item_similarity_jaccard = np.zeros((items,items))
	item_similarity_pearson = np.zeros((items,items))
	for item1 in range(items):
		print(item1)
		for item2 in range(items):
			if np.count_nonzero(data[:,item1]) and np.count_nonzero(data[:,item2]):
				item_similarity_cosine[item1][item2] = 1-scipy.spatial.distance.cosine(data[:,item1],data[:,item2])
				item_similarity_jaccard[item1][item2] = 1-scipy.spatial.distance.jaccard(data[:,item1],data[:,item2])
				try:
					if not math.isnan(scipy.stats.pearsonr(data[:,item1],data[:,item2])[0]):
						item_similarity_pearson[item1][item2] = scipy.stats.pearsonr(data[:,item1],data[:,item2])[0]
					else:
						item_similarity_pearson[item1][item2] = 0
				except:
					item_similarity_pearson[item1][item2] = 0

			#f_i_d.write(str(item1) + "," + str(item2) + "," + str(item_similarity_cosine[item1][item2]) + "," + str(item_similarity_jaccard[item1][item2]) + "," + str(item_similarity_pearson[item1][item2]) + "\n")
	#f_i_d.close()
	return item_similarity_cosine, item_similarity_jaccard, item_similarity_pearson

def crossValidation(data):
	k_fold = KFold(n_splits=10)

	Mat = np.zeros((users,items))
	for e in data:
		Mat[e[0]-1][e[1]-1] = e[2]

	sim_item_cosine, sim_item_jaccard, sim_item_pearson = similarity_item(Mat)
	#sim_item_cosine, sim_item_jaccard, sim_item_pearson = np.random.rand(items,items), np.random.rand(items,items), np.random.rand(items,items) 

	'''sim_item_cosine = np.zeros((items,items))
	sim_item_jaccard = np.zeros((items,items))
	sim_item_pearson = np.zeros((items,items))

	f_sim_i = open("sim_item_based.txt", "r")
	for row in f_sim_i:
		r = row.strip().split(',')
		sim_item_cosine[int(r[0])][int(r[1])] = float(r[2])
		sim_item_jaccard[int(r[0])][int(r[1])] = float(r[3])
		sim_item_pearson[int(r[0])][int(r[1])] = float(r[4])
	f_sim_i.close()'''

	rmse_cosine = []
	rmse_jaccard = []
	rmse_pearson = []

	for train_indices, test_indices in k_fold:
		train = [data[i] for i in train_indices]
		test = [data[i] for i in test_indices]

		M = np.zeros((users,items))

		for e in train:
			M[e[0]-1][e[1]-1] = e[2]

		true_rate = []
		pred_rate_cosine = []
		pred_rate_jaccard = []
		pred_rate_pearson = []

		for e in test:
			user = e[0]
			item = e[1]
			true_rate.append(e[2])

			pred_cosine = 3.0
			pred_jaccard = 3.0
			pred_pearson = 3.0

			#item-based
			if np.count_nonzero(M[:,item-1]):
				sim_cosine = sim_item_cosine[item-1]
				sim_jaccard = sim_item_jaccard[item-1]
				sim_pearson = sim_item_pearson[item-1]
				ind = (M[user-1] > 0)
				#ind[item-1] = False
				normal_cosine = np.sum(np.absolute(sim_cosine[ind]))
				normal_jaccard = np.sum(np.absolute(sim_jaccard[ind]))
				normal_pearson = np.sum(np.absolute(sim_pearson[ind]))
				if normal_cosine > 0:
					pred_cosine = np.dot(sim_cosine,M[user-1])/normal_cosine

				if normal_jaccard > 0:
					pred_jaccard = np.dot(sim_jaccard,M[user-1])/normal_jaccard

				if normal_pearson > 0:
					pred_pearson = np.dot(sim_pearson,M[user-1])/normal_pearson

			if pred_cosine < 0:
				pred_cosine = 0

			if pred_cosine > 5:
				pred_cosine = 5

			if pred_jaccard < 0:
				pred_jaccard = 0

			if pred_jaccard > 5:
				pred_jaccard = 5

			if pred_pearson < 0:
				pred_pearson = 0

			if pred_pearson > 5:
				pred_pearson = 5

			print(str(user) + "\t" + str(item) + "\t" + str(e[2]) + "\t" + str(pred_cosine) + "\t" + str(pred_jaccard) + "\t" + str(pred_pearson))
			pred_rate_cosine.append(pred_cosine)
			pred_rate_jaccard.append(pred_jaccard)
			pred_rate_pearson.append(pred_pearson)

		rmse_cosine.append(sqrt(mean_squared_error(true_rate, pred_rate_cosine)))
		rmse_jaccard.append(sqrt(mean_squared_error(true_rate, pred_rate_jaccard)))
		rmse_pearson.append(sqrt(mean_squared_error(true_rate, pred_rate_pearson)))

		print(str(sqrt(mean_squared_error(true_rate, pred_rate_cosine))) + "\t" + str(sqrt(mean_squared_error(true_rate, pred_rate_jaccard))) + "\t" + str(sqrt(mean_squared_error(true_rate, pred_rate_pearson))))
		#raw_input()

	#print sum(rms) / float(len(rms))
	rmse_cosine = sum(rmse_cosine) / float(len(rmse_cosine))
	rmse_pearson = sum(rmse_pearson) / float(len(rmse_pearson))
	rmse_jaccard = sum(rmse_jaccard) / float(len(rmse_jaccard))

	print(str(rmse_cosine) + "\t" + str(rmse_jaccard) + "\t" + str(rmse_pearson))

	f_rmse = open("rmse_item.txt","w")
	f_rmse.write(str(rmse_cosine) + "\t" + str(rmse_jaccard) + "\t" + str(rmse_pearson) + "\n")

	rmse = [rmse_cosine, rmse_jaccard, rmse_pearson]
	req_sim = rmse.index(min(rmse))

	print(req_sim)
	f_rmse.write(str(req_sim))
	f_rmse.close()

	if req_sim == 0:
		sim_mat_item = sim_item_cosine

	if req_sim == 1:
		sim_mat_item = sim_item_jaccard

	if req_sim == 2:
		sim_mat_item = sim_item_pearson

	#predictRating(Mat, sim_mat_item)
	return Mat, sim_mat_item


def predictRating(recommend_data):

	M, sim_item = crossValidation(recommend_data)

	#f = open("toBeRated.csv","r")
	f = open(sys.argv[2],"r")
	toBeRated = {"user":[], "item":[]}
	for row in f:
		r = row.split(',')	
		toBeRated["item"].append(int(r[1]))
		toBeRated["user"].append(int(r[0]))

	f.close()

	pred_rate = []

	#fw = open('result2.csv','w')
	fw_w = open('result2.csv','w')

	l = len(toBeRated["user"])
	for e in range(l):
		user = toBeRated["user"][e]
		item = toBeRated["item"][e]

		pred = 3.0

		#item-based
		if np.count_nonzero(M[:,item-1]):
			sim = sim_item[item-1]
			ind = (M[user-1] > 0)
			#ind[item-1] = False
			normal = np.sum(np.absolute(sim[ind]))
			if normal > 0:
				pred = np.dot(sim,M[user-1])/normal

		if pred < 0:
			pred = 0

		if pred > 5:
			pred = 5

		pred_rate.append(pred)
		print(str(user) + "," + str(item) + "," + str(pred))
		#fw.write(str(user) + "," + str(item) + "," + str(pred) + "\n")
		fw_w.write(str(pred) + "\n")

	#fw.close()
	fw_w.close()

recommend_data = readingFile("ratings.csv")
# recommend_data = readingFile(sys.argv[1])
#crossValidation(recommend_data)
predictRating(recommend_data)

Hello
0


KeyboardInterrupt: 

<div class=title2>
    <b><font size='3'>Hybrid</font></b>
</div>

In [ ]:
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
#from sklearn.utils.extmath import np.dot

users = 6040
items = 3952

def readingFile(filename):
	f = open(filename,"r")
	data = []
	for row in f:
		r = row.split(',')
		e = [int(r[0]), int(r[1]), int(r[2])]
		data.append(e)
	return data

def userData(filename):
	f = open(filename,"r")
	data = np.zeros((users,3))
	for row in f:
		r = row.strip().split(',')
		if r[1] == "M" or r[1] == "m":
			data[int(r[0])-1] = [1,(int(r[2])/56.0),((int(r[3])+1.0)/21.0)]
		else:
			data[int(r[0])-1] = [0,(int(r[2])/56.0),((int(r[3])+1)/21.0)]

	return data


def itemData(filename):
	f = open(filename,"r")
	data = np.zeros((items,18))
	genre = {"Action":0, "Adventure":1, "Animation":2, "Children's":3, "Comedy":4, "Crime":5, "Documentary":6, "Drama":7, "Fantasy":8, "Film-Noir":9, "Horror":10, "Musical":11, "Mystery":12, "Romance":13, "Sci-Fi":14, "Thriller":15, "War":16, "Western":17 }
	for row in f:
		r = row.split(',')
		g = r[len(r)-1].split('|')
		for e in g:
			if e.strip() not in genre.keys():
				continue
			else:
				data[int(r[0])-1][genre[e.strip()]] = 1

	return data

def similarity_item(data):
	print("Hello Item")
	#f_i_d = open("sim_item_hybrid.txt","w")
	item_similarity_cosine = np.zeros((items,items))
	item_similarity_jaccard = np.zeros((items,items))
	item_similarity_pearson = np.zeros((items,items))
	for item1 in range(items):
		print(item1)
		for item2 in range(items):
			if np.count_nonzero(data[item1]) and np.count_nonzero(data[item2]):
				item_similarity_cosine[item1][item2] = 1-scipy.spatial.distance.cosine(data[item1],data[item2])
				item_similarity_jaccard[item1][item2] = 1-scipy.spatial.distance.jaccard(data[item1],data[item2])
				try:
					if not math.isnan(scipy.stats.pearsonr(data[item1],data[item2])[0]):
						item_similarity_pearson[item1][item2] = scipy.stats.pearsonr(data[item1],data[item2])[0]
					else:
						item_similarity_pearson[item1][item2] = 0
				except:
					item_similarity_pearson[item1][item2] = 0

			#f_i_d.write(str(item1) + "," + str(item2) + "," + str(item_similarity_cosine[item1][item2]) + "," + str(item_similarity_jaccard[item1][item2]) + "," + str(item_similarity_pearson[item1][item2]) + "\n")
	#f_i_d.close()
	return item_similarity_cosine, item_similarity_jaccard, item_similarity_pearson


def similarity_user(data):
	print("Hello User")
	#f_i_d = open("sim_user_hybrid.txt","w")
	user_similarity_cosine = np.zeros((users,users))
	user_similarity_jaccard = np.zeros((users,users))
	user_similarity_pearson = np.zeros((users,users))
	for user1 in range(users):
		print(user1)
		for user2 in range(users):
			if np.count_nonzero(data[user1]) and np.count_nonzero(data[user2]):
				user_similarity_cosine[user1][user2] = 1-scipy.spatial.distance.cosine(data[user1],data[user2])
				user_similarity_jaccard[user1][user2] = 1-scipy.spatial.distance.jaccard(data[user1],data[user2])
				try:
					if not math.isnan(scipy.stats.pearsonr(data[user1],data[user2])[0]):
						user_similarity_pearson[user1][user2] = scipy.stats.pearsonr(data[user1],data[user2])[0]
					else:
						user_similarity_pearson[user1][user2] = 0
				except:
					user_similarity_pearson[user1][user2] = 0

			#f_i_d.write(str(user1) + "," + str(user2) + "," + str(user_similarity_cosine[user1][user2]) + "," + str(user_similarity_jaccard[user1][user2]) + "," + str(user_similarity_pearson[user1][user2]) + "\n")
	#f_i_d.close()
	return user_similarity_cosine, user_similarity_jaccard, user_similarity_pearson

def crossValidation(data, user_data, item_data):
	k_fold = KFold(n_splits=10)

	sim_user_cosine, sim_user_jaccard, sim_user_pearson = similarity_user(user_data)
	sim_item_cosine, sim_item_jaccard, sim_item_pearson = similarity_item(item_data)
	#sim_user_cosine, sim_user_jaccard, sim_user_pearson = np.random.rand(users,users), np.random.rand(users,users), np.random.rand(users,users)
	#sim_item_cosine, sim_item_jaccard, sim_item_pearson = np.random.rand(items,items), np.random.rand(items,items), np.random.rand(items,items) 

	'''sim_user_cosine = np.zeros((users,users))
	sim_user_jaccard = np.zeros((users,users))
	sim_user_pearson = np.zeros((users,users))

	f_sim = open("sim_user_hybrid.txt", "r")
	for row in f_sim:
		#print row
		r = row.strip().split(',')
		sim_user_cosine[int(r[0])][int(r[1])] = float(r[2])
		sim_user_jaccard[int(r[0])][int(r[1])] = float(r[3])
		sim_user_pearson[int(r[0])][int(r[1])] = float(r[4])
	f_sim.close()


	sim_item_cosine = np.zeros((items,items))
	sim_item_jaccard = np.zeros((items,items))
	sim_item_pearson = np.zeros((items,items))

	f_sim_i = open("sim_item_hybrid.txt", "r")
	for row in f_sim_i:
		#print row
		r = row.strip().split(',')
		sim_item_cosine[int(r[0])][int(r[1])] = float(r[2])
		sim_item_jaccard[int(r[0])][int(r[1])] = float(r[3])
		sim_item_pearson[int(r[0])][int(r[1])] = float(r[4])
	f_sim_i.close()'''

	rmse_cosine = []
	rmse_jaccard = []
	rmse_pearson = []

	for train_indices, test_indices in k_fold:
		train = [data[i] for i in train_indices]
		test = [data[i] for i in test_indices]

		M = np.zeros((users,items))

		for e in train:
			M[e[0]-1][e[1]-1] = e[2]

		true_rate = []
		pred_rate_cosine = []
		pred_rate_jaccard = []
		pred_rate_pearson = []

		for e in test:
			user = e[0]
			item = e[1]
			true_rate.append(e[2])

			user_pred_cosine = 3.0
			item_pred_cosine = 3.0

			user_pred_jaccard = 3.0
			item_pred_jaccard = 3.0

			user_pred_pearson = 3.0
			item_pred_pearson = 3.0

			#item-based
			if np.count_nonzero(M[:,item-1]):
				sim_cosine = sim_item_cosine[item-1]
				sim_jaccard = sim_item_jaccard[item-1]
				sim_pearson = sim_item_pearson[item-1]
				ind = (M[user-1] > 0)
				#ind[item-1] = False
				normal_cosine = np.sum(np.absolute(sim_cosine[ind]))
				normal_jaccard = np.sum(np.absolute(sim_jaccard[ind]))
				normal_pearson = np.sum(np.absolute(sim_pearson[ind]))
				if normal_cosine > 0:
					item_pred_cosine = np.dot(sim_cosine,M[user-1])/normal_cosine

				if normal_jaccard > 0:
					item_pred_jaccard = np.dot(sim_jaccard,M[user-1])/normal_jaccard

				if normal_pearson > 0:
					item_pred_pearson = np.dot(sim_pearson,M[user-1])/normal_pearson

			if item_pred_cosine < 0:
				item_pred_cosine = 0

			if item_pred_cosine > 5:
				item_pred_cosine = 5

			if item_pred_jaccard < 0:
				item_pred_jaccard = 0

			if item_pred_jaccard > 5:
				item_pred_jaccard = 5

			if item_pred_pearson < 0:
				item_pred_pearson = 0

			if item_pred_pearson > 5:
				item_pred_pearson = 5

			#user-based
			if np.count_nonzero(M[user-1]):
				sim_cosine = sim_user_cosine[user-1]
				sim_jaccard = sim_user_jaccard[user-1]
				sim_pearson = sim_user_pearson[user-1]
				ind = (M[:,item-1] > 0)
				#ind[user-1] = False
				normal_cosine = np.sum(np.absolute(sim_cosine[ind]))
				normal_jaccard = np.sum(np.absolute(sim_jaccard[ind]))
				normal_pearson = np.sum(np.absolute(sim_pearson[ind]))
				if normal_cosine > 0:
					user_pred_cosine = np.dot(sim_cosine,M[:,item-1])/normal_cosine

				if normal_jaccard > 0:
					user_pred_jaccard = np.dot(sim_jaccard,M[:,item-1])/normal_jaccard

				if normal_pearson > 0:
					user_pred_pearson = np.dot(sim_pearson,M[:,item-1])/normal_pearson

			if user_pred_cosine < 0:
				user_pred_cosine = 0

			if user_pred_cosine > 5:
				user_pred_cosine = 5

			if user_pred_jaccard < 0:
				user_pred_jaccard = 0

			if user_pred_jaccard > 5:
				user_pred_jaccard = 5

			if user_pred_pearson < 0:
				user_pred_pearson = 0

			if user_pred_pearson > 5:
				user_pred_pearson = 5

			if (user_pred_cosine != 0 and user_pred_cosine != 5) and (item_pred_cosine != 0 and item_pred_cosine != 5):
				pred_cosine = (user_pred_cosine + item_pred_cosine)/2
			else:
				if (user_pred_cosine == 0 or user_pred_cosine == 5):
					if (item_pred_cosine != 0 and item_pred_cosine != 5):
						pred_cosine = item_pred_cosine
					else:
						pred_cosine = 3.0
				else:
					if (user_pred_cosine != 0 and user_pred_cosine != 5):
						pred_cosine = user_pred_cosine
					else:
						pred_cosine = 3.0

			if (user_pred_jaccard != 0 and user_pred_jaccard != 5) and (item_pred_jaccard != 0 and item_pred_jaccard != 5):
				pred_jaccard = (user_pred_jaccard + item_pred_jaccard)/2
			else:
				if (user_pred_jaccard == 0 or user_pred_jaccard == 5):
					if (item_pred_jaccard != 0 and item_pred_jaccard != 5):
						pred_jaccard = item_pred_jaccard
					else:
						pred_jaccard = 3.0
				else:
					if (user_pred_jaccard != 0 and user_pred_jaccard != 5):
						pred_jaccard = user_pred_jaccard
					else:
						pred_jaccard = 3.0

			if (user_pred_pearson != 0 and user_pred_pearson != 5) and (item_pred_pearson != 0 and item_pred_pearson != 5):
				pred_pearson = (user_pred_pearson + item_pred_pearson)/2
			else:
				if (user_pred_pearson == 0 or user_pred_pearson == 5):
					if (item_pred_pearson != 0 and item_pred_pearson != 5):
						pred_pearson = item_pred_pearson
					else:
						pred_pearson = 3.0
				else:
					if (user_pred_pearson != 0 and user_pred_pearson != 5):
						pred_pearson = user_pred_pearson
					else:
						pred_pearson = 3.0
			
			#pred_cosine = (user_pred_cosine + item_pred_cosine)/2
			#pred_jaccard = (user_pred_jaccard + item_pred_jaccard)/2
			#pred_pearson = (user_pred_pearson + item_pred_pearson)/2
			print(str(user) + "\t" + str(item) + "\t" + str(e[2]) + "\t" + str(pred_cosine) + "\t" + str(pred_jaccard) + "\t" + str(pred_pearson))
			pred_rate_cosine.append(pred_cosine)
			pred_rate_jaccard.append(pred_jaccard)
			pred_rate_pearson.append(pred_pearson)

		#print len(true_rate)
		#print len(pred_rate_cosine)
		rmse_cosine.append(sqrt(mean_squared_error(true_rate, pred_rate_cosine)))
		rmse_jaccard.append(sqrt(mean_squared_error(true_rate, pred_rate_jaccard)))
		rmse_pearson.append(sqrt(mean_squared_error(true_rate, pred_rate_pearson)))

		print(str(sqrt(mean_squared_error(true_rate, pred_rate_cosine))) + "\t" + str(sqrt(mean_squared_error(true_rate, pred_rate_jaccard))) + "\t" + str(sqrt(mean_squared_error(true_rate, pred_rate_pearson))))
		#raw_input()

	#print sum(rms) / float(len(rms))
	rmse_cosine = sum(rmse_cosine) / float(len(rmse_cosine))
	rmse_pearson = sum(rmse_pearson) / float(len(rmse_pearson))
	rmse_jaccard = sum(rmse_jaccard) / float(len(rmse_jaccard))

	print(str(rmse_cosine) + "\t" + str(rmse_jaccard) + "\t" + str(rmse_pearson))

	f_rmse = open("rmse_hybrid.txt","w")
	f_rmse.write(str(rmse_cosine) + "\t" + str(rmse_jaccard) + "\t" + str(rmse_pearson) + "\n")

	rmse = [rmse_cosine, rmse_jaccard, rmse_pearson]
	req_sim = rmse.index(min(rmse))

	print(req_sim)
	f_rmse.write(str(req_sim))
	f_rmse.close()

	if req_sim == 0:
		sim_mat_user = sim_user_cosine
		sim_mat_item = sim_item_cosine

	if req_sim == 1:
		sim_mat_user = sim_user_jaccard
		sim_mat_item = sim_item_jaccard

	if req_sim == 2:
		sim_mat_user = sim_user_pearson
		sim_mat_item = sim_item_pearson

	#predictRating(data, sim_mat_user, sim_mat_item)
	return sim_mat_user, sim_mat_item


def predictRating(data, user_data, item_data):

	sim_user, sim_item = crossValidation(data, user_data, item_data)

	M = np.zeros((users,items))
	for e in data:
		M[e[0]-1][e[1]-1] = e[2]

	#f = open("toBeRated.csv","r")
	f = open(sys.argv[2],"r")	
	toBeRated = {"user":[], "item":[]}
	for row in f:
		r = row.split(',')	
		toBeRated["item"].append(int(r[1]))
		toBeRated["user"].append(int(r[0]))

	f.close()

	pred_rate = []

	#fw = open('result3.csv','w')
	fw_w = open('result3.csv','w')

	l = len(toBeRated["user"])
	for e in range(l):
		user = toBeRated["user"][e]
		item = toBeRated["item"][e]

		user_pred = 3.0
		item_pred = 3.0

		#item-based
		if np.count_nonzero(M[:,item-1]):
			sim = sim_item[item-1]
			ind = (M[user-1] > 0)
			#ind[item-1] = False
			normal = np.sum(np.absolute(sim[ind]))
			if normal > 0:
				item_pred = np.dot(sim,M[user-1])/normal

		if item_pred < 0:
			item_pred = 0

		if item_pred > 5:
			item_pred = 5

		#user-based
		if np.count_nonzero(M[user-1]):
			sim = sim_user[user-1]
			ind = (M[:,item-1] > 0)
			#ind[user-1] = False
			normal = np.sum(np.absolute(sim[ind]))
			if normal > 0:
				user_pred = np.dot(sim,M[:,item-1])/normal

		if user_pred < 0:
			user_pred = 0

		if user_pred > 5:
			user_pred = 5

		if (user_pred != 0 and user_pred != 5) and (item_pred != 0 and item_pred != 5):
				pred = (user_pred + item_pred)/2
		else:
			if (user_pred == 0 or user_pred == 5):
				if (item_pred != 0 and item_pred != 5):
					pred = item_pred
				else:
					pred = 3.0
			else:
				if (user_pred != 0 and user_pred != 5):
					pred = user_pred
				else:
					pred = 3.0

		#pred = (user_pred + item_pred)/2
		pred_rate.append(pred)
		print(str(user) + "," + str(item) + "," + str(pred))
		#fw.write(str(user) + "," + str(item) + "," + str(pred) + "\n")
		fw_w.write(str(pred) + "\n")

	#fw.close()
	fw_w.close()

recommend_data = readingFile("ratings.csv")
# recommend_data = readingFile(sys.argv[1])
user_data = userData("users.csv")
item_data = itemData("movies.csv")
predictRating(recommend_data, user_data, item_data)
#crossValidation(recommend_data, user_data, item_data)

Hello User
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274